In [ ]:
# Install semantic link and CAT tool
!pip install semantic-link --q
!pip install "https://raw.githubusercontent.com/m-kovalsky/fabric_cat_tools/main/fabric_cat_tools-0.3.0-py3-none-any.whl" --q

In [ ]:
# Import packages
from pyspark.sql.functions import current_timestamp, concat_ws, lit, col, lpad, date_format, dayofweek, year, quarter, month, date_add, datediff, concat
from pyspark.sql.types import StructType, StructField, StringType, BooleanType
from pyspark.sql.functions import current_timestamp
from delta.tables import DeltaTable
import fabric_cat_tools as fct
import sempy.fabric as fabric
import re
import datetime
import time
import pandas as pd

In [ ]:
# Monitoring lakehouse
lucid_monitor_lakehouse = 'LucidBI'

try:
    # Retrieve the list of lakehouses
    df_lakehouse = fct.list_lakehouses()
    lakehouse_names = df_lakehouse['Lakehouse Name'].tolist()

    # Check if the lakehouse exists
    if lucid_monitor_lakehouse in lakehouse_names:
        print(f'Lakehouse {lucid_monitor_lakehouse} already exists. Skipping creation.')
        
        # Retrieve the lakehouse details from the dataframe
        lakehouse_info = df_lakehouse[df_lakehouse['Lakehouse Name'] == lucid_monitor_lakehouse].iloc[0]
    else:
        # Create monitor lakehouse if it does not exist
        fabric.create_lakehouse(display_name=lucid_monitor_lakehouse)
        print(f'Lakehouse {lucid_monitor_lakehouse} created.')
        
        # Refresh the list of lakehouses to get updated info
        df_lakehouse = fct.list_lakehouses()
        lakehouse_info = df_lakehouse[df_lakehouse['Lakehouse Name'] == lucid_monitor_lakehouse].iloc[0]

    # Capture Tables Path and Files Path
    lakehouse_id = lakehouse_info['Lakehouse ID']
    workspace_id = fabric.get_workspace_id()
    lakehouse_table_path = f'abfss://{workspace_id}@onelake.dfs.fabric.microsoft.com/{lakehouse_id}/Tables'
    lakehouse_file_path = f'abfss://{workspace_id}@onelake.dfs.fabric.microsoft.com/{lakehouse_id}/Files'
    print(f'Table and File paths captured')

except Exception as e:
    print(f'An unexpected error occurred while processing the lakehouse: {e}')

#### Update with location and name of your Fabric Capacity Metrics app

```
# Define the location of the metrics application
metrics_app_model = 'Fabric Capacity Metrics'
metrics_app_workspace = 'Microsoft Fabric Capacity Metrics'
```

In [ ]:
# Define the location of the metrics application
try:
    metrics_app_model = 'Fabric Capacity Metrics'
    metrics_app_workspace = 'Microsoft Fabric Capacity Metrics'
except Exception as exc:
    print(f'Update with your capacity metrics app details: {exc}')

In [ ]:
# Trigger refresh of the capacity metrics semantic model
refresh_id = fabric.refresh_dataset(metrics_app_model, metrics_app_workspace)

def poll_refresh_status(model, refresh_id, workspace, timeout=120, interval=5):
    """
    Polls the status of a refresh operation until completion or failure.
    Returns the final execution details.
    """
    start_time = time.time()
    while True:
        # Check the status of the refresh operation
        details = fabric.get_refresh_execution_details(model, refresh_id, workspace)
        status = details.status

        print(f'Current status: {status}')

        # Exit loop if the operation has completed or failed
        if status in ['Completed', 'Failed']:
            print(f'Refresh operation finished with status: {status}')
            return details

        # Check for timeout
        if time.time() - start_time > timeout:
            print('Timeout reached, exiting status check.')
            return details

        # Wait briefly before the next status check
        time.sleep(interval)

# Execute the refresh and capture logs
execution_details = poll_refresh_status(metrics_app_model, refresh_id, metrics_app_workspace)

# Define logging attributes
log_attributes = [
    'start_time', 'end_time', 'type', 'commit_mode', 
    'status', 'extended_status', 'current_refresh_type', 'number_of_attempts'
]

# Create a logging dictionary from execution details
execution_log = {attr: getattr(execution_details, attr, None) for attr in log_attributes}

# Convert refresh details to a pandas DataFrame
df_logging = pd.DataFrame([execution_log])

# Convert to a Spark DataFrame
df_logging = spark.createDataFrame(df_logging)

# Add tagging columns to the DataFrame
df_logging = (df_logging
              .withColumn('capacity_metrics_workspace', lit(metrics_app_workspace))
              .withColumn('capacity_metrics_model', lit(metrics_app_model)))

# Define the table name and directory path
table = 'log_refresh_execution'
directory = f'{lakehouse_file_path}/capacity_metrics/{table}'
file_path = f"{directory}/{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

# Write the DataFrame to a parquet file
df_logging.write.format('parquet').save(file_path)

# Write the DataFrame to a Delta table
df_logging.write.format('delta').mode('append').save(f'{lakehouse_table_path}/{table}')

In [ ]:
import re
import datetime
from pyspark.sql.functions import current_timestamp

def clean_and_lowercase(name):
    """
    Remove special characters from the name, replace them with underscores, trim leading and trailing underscores,
    and convert the result to lowercase.
    """
    # Replace special characters with underscores
    name_with_underscores = re.sub(r'\W+', '_', name)

    # Trim leading/trailing underscores and convert to lowercase
    cleaned_name = re.sub(r'^_+|_+$', '', name_with_underscores).lower()
    
    return cleaned_name

def parse_header(name):
    """
    Extract text within brackets, remove special characters, then convert to lowercase.
    If no brackets are found, clean the name directly.
    """
    # Search for text within brackets
    match = re.search(r'\[(.*?)\]', name)
    
    # Use found text or the whole name if no brackets are found
    return clean_and_lowercase(match.group(1)) if match else clean_and_lowercase(name)

def evaluate_and_write_table(table_name):
    """
    Evaluate a DAX query on the specified table, clean column names, convert the DataFrame to Spark format,
    and write it to parquet and delta formats.
    """
    # Evaluate table using DAX and the Fabric Capacity model
    df_table = fabric.evaluate_dax(metrics_app_model, f"EVALUATE '{table_name}'", metrics_app_workspace)
    
    # Clean up column names
    df_table.columns = [parse_header(col) for col in df_table.columns]
    
    # Clean table name for use in paths
    clean_table_name = f'stage_{clean_and_lowercase(table_name)}'
    
    # Define file storage directory
    table_directory = f'{lakehouse_file_path}/capacity_metrics/{clean_table_name}'
    
    # File path with timestamp
    file_path = f"{table_directory}/{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

    # Convert DataFrame to Spark format
    df_table = spark.createDataFrame(df_table)
    
    # Save DataFrame in parquet format
    df_table.write.format('parquet').save(file_path)
    
    # Save DataFrame in delta format
    df_table.write.format('delta').mode('overwrite').save(f'{lakehouse_table_path}/{clean_table_name}')

    # Create temp view
    df_table.createOrReplaceTempView(f'vw_{clean_table_name}')

# List tables in the model and workspace
df_tables = fabric.list_tables(metrics_app_model, workspace=metrics_app_workspace)

# Extract table names
table_names = df_tables['Name'].tolist()

# Include only specific tables
include_tables = set([
    'MetricsByItemandOperationandHour',
    'StorageByWorkspacesandHour',
    'CUDetail',
    'MaxMemoryByItemAndHour'
])

# Filter table names to include only specified tables
filtered_table_names = [table_name for table_name in table_names if table_name in include_tables]

# Process each specified table
for table_name in filtered_table_names:
    try:
        evaluate_and_write_table(table_name)
        print(f'{table_name} completed successfully.')
    except Exception as exc:
        print(f'Error processing {table_name}: {exc}')


In [ ]:
from pyspark.sql.functions import current_timestamp
from delta.tables import DeltaTable

def upsert_to_table(config):
    """
    Implements a generic upsert pattern for loading data into a dimensional layer in a Delta table.
    Maintains audit fields for data insertion and updates.
    
    Written by: Will Crayger of Lucid
    """
    
    table_name = config['table_name']
    lakehouse_table_path = config['lakehouse_table_path']
    df_source = config['source_dataframe']
    match_keys = config['match_keys']
    
    full_table_path = f'{lakehouse_table_path}/{table_name}'
    
    # Capture the current timestamp for audit fields
    current_ts = current_timestamp()

    try:
        # Check if the Delta table exists and get the DeltaTable object
        if DeltaTable.isDeltaTable(spark, full_table_path):
            deltaTable = DeltaTable.forPath(spark, full_table_path)
        else:
            print(f'Table {table_name} does not exist. Creating it.')

            # Add audit columns
            df_source = df_source \
                    .withColumn('inserted_date_time', current_ts) \
                    .withColumn('updated_date_time', current_ts)

            df_source.write.format('delta').mode('append').save(full_table_path)

            print(f'Upsert for {table_name} complete.')
            return            

        # Enhance df_source with necessary audit fields
        df_source = (df_source
                    .withColumn('inserted_date_time', current_ts)
                    .withColumn('updated_date_time', current_ts))
        
        # Define the match condition for the MERGE operation
        match_condition = ' AND '.join([f'target.{k} = source.{k}' for k in match_keys])

        # Generate the list of excluded columns for the MERGE operation
        excluded_columns = match_keys + ['inserted_date_time', 'updated_date_time']

        # Get non-match key fields excluding audit fields
        non_match_keys = [col for col in df_source.columns if col not in excluded_columns]
        
        # Create a condition to check if any non-match key field has changed
        update_condition = ' OR '.join([f'target.{field} <> source.{field}' for field in non_match_keys])

        # Configure the MERGE operation to update existing records and insert new records
        merge_operation = deltaTable.alias('target') \
                        .merge(df_source.alias('source'), match_condition) \
                        .whenMatchedUpdateAll(condition=update_condition) \
                        .whenNotMatchedInsertAll()
        
        # Execute the merge operation
        merge_operation.execute()

    except Exception as e:
        print(f'An error occurred in upsert_to_delta_table for table {table_name}: {e}')
        raise

In [ ]:
# Define the schema for the workspace DataFrame
schema = StructType([
    StructField('id', StringType(), True),
    StructField('is_read_only', BooleanType(), True),
    StructField('is_on_dedicated_capacity', BooleanType(), True),
    StructField('capacity_id', StringType(), True),
    StructField('default_dataset_storage_format', StringType(), True),
    StructField('type', StringType(), True),
    StructField('name', StringType(), True),
])

try:
    # Fetch workspace details from the fabric service
    df_stage = fabric.list_workspaces()
    
    # Convert the fetched data into a Spark DataFrame using the defined schema
    df_ws_stage_spark = spark.createDataFrame(df_stage, schema=schema)
    
    # Define the table name for storing workspace data
    table_name = 'stage_workspaces'

    # Define file storage directory
    file_directory = f'{lakehouse_file_path}/capacity_metrics/{table_name}'
    
    # File path with timestamp
    file_path = f"{file_directory}/{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

    # Save DataFrame in parquet format
    df_ws_stage_spark.write.format('parquet').save(file_path)

    # Write the DataFrame to a Delta table with an overwrite mode
    df_ws_stage_spark.write.format('delta').mode('overwrite').save(f'{lakehouse_table_path}/{table_name}')
    print(f'{table_name} staged successfully.')

    # Create temp view
    df_ws_stage_spark.createOrReplaceTempView(f'vw_{table_name}')

except Exception as e:
    print(f'An unexpected error occurred while capturing workspace details: {e}')


In [ ]:
# Define the schema for the capacity DataFrame
schema = StructType([
    StructField('id', StringType(), True),
    StructField('display_name', StringType(), True),
    StructField('sku', StringType(), True),
    StructField('region', StringType(), True),
    StructField('state', StringType(), True),
])

try:
    # Fetch capacity details from the fabric service
    df_stage = fabric.list_capacities()
    
    # Convert the raw data to a Spark DataFrame using the predefined schema
    df_stage_spark = spark.createDataFrame(df_stage, schema=schema)
    
    # Define the table name for storing capacity data
    table_name = 'stage_capacities'

    # Define file storage directory
    file_directory = f'{lakehouse_file_path}/capacity_metrics/{table_name}'
    
    # File path with timestamp
    file_path = f"{file_directory}/{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

    # Save DataFrame in parquet format
    df_stage_spark.write.format('parquet').save(file_path)

    # Write the DataFrame to a Delta table with an overwrite mode
    df_stage_spark.write.format('delta').mode('overwrite').save(f'{lakehouse_table_path}/{table_name}')
    print(f'{table_name} staged successfully.')

    # Create temp view
    df_stage_spark.createOrReplaceTempView(f'vw_{table_name}')

except Exception as e:
    print(f'An unexpected error occurred while capturing capacity details: {e}')


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType

# Define the schema for the item DataFrame
schema = StructType([
    StructField('id', StringType(), True),
    StructField('display_name', StringType(), True),
    StructField('description', StringType(), True),
    StructField('type', StringType(), True),
    StructField('workspace_id', StringType(), True),
])

try:
    # Fetch item details from the fabric service
    df_item = fabric.list_items()
    
    # Filter workspaces on dedicated capacity and collect their IDs
    df_workspaces = df_ws_stage_spark.filter(df_ws_stage_spark['is_on_dedicated_capacity'] == 'true').select('id').collect()
    
    # Initialize an empty DataFrame to aggregate items from each workspace
    df_items_all = None

    # Aggregate items from each workspace with dedicated capacity
    for ws in df_workspaces:
        ws_id = ws['id']
        df_items = spark.createDataFrame(fabric.list_items(workspace=ws_id), schema=schema)
        
        # Union the current items DataFrame with the aggregated items DataFrame
        df_stage_spark = df_items if df_items_all is None else df_items_all.union(df_items)

    # Define the table name for storing item data
    table_name = 'stage_items'

    # Define file storage directory
    file_directory = f'{lakehouse_file_path}/capacity_metrics/{table_name}'
    
    # File path with timestamp
    file_path = f"{file_directory}/{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

    # Save DataFrame in parquet format
    df_stage_spark.write.format('parquet').save(file_path)

    # Write the DataFrame to a Delta table with an overwrite mode
    df_stage_spark.write.format('delta').mode('overwrite').save(f'{lakehouse_table_path}/{table_name}')
    print(f'{table_name} staged successfully.')

    # Create temp view
    df_stage_spark.createOrReplaceTempView(f'vw_{table_name}')

except Exception as e:
    print(f'An unexpected error occurred while capturing item details: {e}')

In [ ]:
try:
    # Set table name
    table_name = 'stage_capacity_cost'

    # Check if the delta table already exists
    if DeltaTable.isDeltaTable(spark, f'{lakehouse_table_path}/{table_name}'):
        print(f'Table {table_name} already exists. Skipping load.')
    else:
        # Set github file path
        url_github = 'https://raw.githubusercontent.com/Lucid-Will/Lucid-Capacity-Monitoring/main/supporting_data_files/capacity_cost_by_region/capacity_cost_by_region.csv'
        
        # Load capacity cost file
        pd_stage = pd.read_csv(url_github)
        df_stage_spark = spark.createDataFrame(pd_stage)
        
        # Define file storage directory
        file_directory = f'{lakehouse_file_path}/capacity_metrics/{table_name}'
        
        # File path with timestamp
        file_path = f"{file_directory}/{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

        # Save DataFrame in parquet format
        df_stage_spark.write.format('parquet').save(file_path)

        # Write the DataFrame to a Delta table with an overwrite mode
        df_stage_spark.write.format('delta').mode('overwrite').save(f'{lakehouse_table_path}/{table_name}')
        print(f'{table_name} staged successfully.')

        # Create temp view
        df_stage_spark.createOrReplaceTempView(f'vw_{table_name}')

except Exception as e:
    print(f'An unexpected error occurred while capturing capacity details: {e}')


In [ ]:
try:
    # Set table name
    table_name = 'stage_storage_cost'

    # Check if the delta table already exists
    if DeltaTable.isDeltaTable(spark, f'{lakehouse_table_path}/{table_name}'):
        print(f'Table {table_name} already exists. Skipping load.')
    else:
        # Set github file path
        url_github = 'https://raw.githubusercontent.com/Lucid-Will/Lucid-Capacity-Monitoring/main/supporting_data_files/onelake_storage_by_region/onelake_storage_by_region.csv'
        
        # Load storage cost file
        pd_stage = pd.read_csv(url_github)
        df_stage_spark = spark.createDataFrame(pd_stage)
        
        # Define file storage directory
        file_directory = f'{lakehouse_file_path}/capacity_metrics/{table_name}'
        
        # File path with timestamp
        file_path = f"{file_directory}/{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

        # Save DataFrame in parquet format
        df_stage_spark.write.format('parquet').save(file_path)

        # Write the DataFrame to a Delta table with an overwrite mode
        df_stage_spark.write.format('delta').mode('overwrite').save(f'{lakehouse_table_path}/{table_name}')
        print(f'{table_name} staged successfully.')

        # Create temp view
        df_stage_spark.createOrReplaceTempView(f'vw_{table_name}')

except Exception as e:
    print(f'An unexpected error occurred while capturing storage details: {e}')


In [ ]:
try:
    # Assign table name
    table_name = 'stage_date'

    # Check if the date table already exists
    if DeltaTable.isDeltaTable(spark, f'{lakehouse_table_path}/{table_name}'):
        print(f'Table {table_name} already exists. Skipping creation.')
    else:
        print(f'Table {table_name} does not exist. Creating it.')

        # Define your date range
        start_date = '1901-01-01'
        end_date = '2100-12-31'

        # Create a DataFrame with the date range difference in days
        date_diff = spark.createDataFrame([(start_date, end_date)], ['start', 'end'])
        date_diff = date_diff.withColumn('diff', datediff(col('end'), col('start'))).collect()[0]['diff']

        # Generate a DataFrame with the sequence of dates
        date_range_diff = spark.range(0, date_diff + 1).withColumnRenamed('id', 'day_id')
        start_date_df = spark.createDataFrame([(start_date,)], ['start_date'])

        # Cast 'day_id' to integer and create a sequence of dates by adding it to 'start_date'
        df_date = start_date_df.crossJoin(date_range_diff) \
            .select(date_add(col('start_date'), col('day_id').cast('int')).alias('date'))

        # Enrich the DataFrame with date attributes
        df_date = df_date.select(
            date_format(col('date'), 'yyyyMMdd').cast('int').alias('calendar_date_key'),
            col('date').alias('calendar_date'),
            dayofweek(col('date')).cast('int').alias('calendar_weekday_number'),
            date_format(col('date'), 'EEEE').alias('calendar_weekday'),
            lpad(month(col('date')).cast('string'), 2, '0').alias('calendar_month_number'),
            date_format(col('date'), 'MMMM').alias('calendar_month'),
            concat(year(col('date')), lpad(month(col('date')).cast('string'), 2, '0')).alias('calendar_year_month_number'),
            date_format(col('date'), 'MMMM yyyy').alias('calendar_month_year'),
            quarter(col('date')).cast('string').alias('calendar_quarter_number'),
            concat(lit('Q'), quarter(col('date')).cast('string')).alias('calendar_quarter'),
            year(col('date')).alias('calendar_year_number')
        ).distinct()

        # Write to delta table
        df_date.write.format('delta').save(f'{lakehouse_table_path}/{table_name}')
        print(f'Table {table_name} created.')

        # Create temp view
        df_date.createOrReplaceTempView(f'vw_{table_name}')

except Exception as e:
    print(f'An unexpected error occurred while capturing date details: {e}')


In [ ]:
# Assign measure table name
table_name = 'measure_table'

try:
    # Check if the measure table already exists
    if DeltaTable.isDeltaTable(spark, f'{lakehouse_table_path}/{table_name}'):
        print(f'Table {table_name} already exists. Skipping creation.')
    else:
        # Create measure table shell since it does not exist
        print(f'Table {table_name} does not exist. Creating it.')

        df_measure = spark.sql("""
            SELECT 1 AS Value
        """)

        # Write the DataFrame to a delta table
        df_measure.write.format('delta').save(f'{lakehouse_table_path}/{table_name}')

        print(f'Table {table_name} created.')

except Exception as e:
    print(f'An unexpected error occurred while creating {table_name}: {e}')

In [ ]:
# Stage and load workspace
df_workspace = spark.sql("""
    SELECT UPPER(id)    workspace_id,
           name         workspace_name,
           capacity_id
    FROM   vw_stage_workspaces
    WHERE  capacity_id IS NOT NULL
""")

# Define configuration for the upsert operation
config = {
    'table_name': 'dim_workspace',  # Destination table for the upsert
    'lakehouse_table_path': lakehouse_table_path, # Table path for writing delta
    'source_dataframe': df_workspace,  # DataFrame to upsert
    'match_keys': ['workspace_id']  # Key columns to match for upsert
}

# Execute upsert to the dimensional table
upsert_to_table(config)

In [ ]:
# Stage and load capacity
df_capacity = spark.sql("""
    WITH storage AS (
        SELECT region, cost
        FROM   vw_stage_storage_cost
        WHERE  storage_type = 'OneLake storage'
    ),
    bcdr AS (
        SELECT region, cost
        FROM   vw_stage_storage_cost
        WHERE  storage_type = 'OneLake BCDR storage'
    ),
    cache AS (
        SELECT region, cost
        FROM   vw_stage_storage_cost
        WHERE  storage_type = 'OneLake cache'
    )
    SELECT DISTINCT 
           UPPER(a.id)         capacity_id,
           a.display_name      capacity_name,
           a.sku,
           a.region,
           b.capacity_unit,
           b.pay_go_hour,
           b.reservation_hour,
           c.cost              onelake_storage_cost,
           d.cost              onelake_bcdr_storage_cost,
           e.cost              onelake_cache_cost,
           a.state
    FROM   vw_stage_capacities a
    LEFT JOIN vw_stage_capacity_cost b
        ON a.region = b.region
       AND a.sku = b.sku
    LEFT JOIN storage c
        ON a.region = c.region
    LEFT JOIN bcdr d
        ON a.region = d.region
    LEFT JOIN cache e
        ON a.region = e.region
    WHERE  capacity_unit IS NOT NULL
""")

# Define configuration for upsert operation
config = {
    'table_name': 'dim_capacity',  # Destination table for the upsert
    'lakehouse_table_path': lakehouse_table_path, # Table path for writing delta
    'source_dataframe': df_capacity,  # DataFrame to upsert
    'match_keys': ['capacity_id']  # Key columns to match for upsert
}

# Create temp view
df_capacity.createOrReplaceTempView("vw_dim_capacity")

# Execute upsert to the dimensional table
upsert_to_table(config)

In [ ]:
# Stage and load item
df_item = spark.sql("""
    SELECT UPPER(id)            item_id,
           UPPER(workspace_id)  workspace_id,
           display_name         item_name,
           type                 item_type
    FROM   vw_stage_items
    ORDER BY item_name ASC
""")

# Define configuration for the upsert operation
config = {
    'table_name': 'dim_item',  # Destination table for the upsert
    'lakehouse_table_path': lakehouse_table_path, # Table path for writing delta
    'source_dataframe': df_item,  # DataFrame to upsert
    'match_keys': ['item_id']  # Key columns to match for upsert
}

# Execute upsert to the dimensional table
upsert_to_table(config)

In [ ]:
# Stage and load date
df_date = spark.sql("""
    SELECT DISTINCT d.*
    FROM   vw_stage_date d
    INNER JOIN vw_stage_metricsbyitemandoperationandhour m 
        ON d.calendar_date = m.date
""")

# Define configuration for the upsert operation
config = {
    'table_name': 'dim_date',  # Destination table for the upsert
    'lakehouse_table_path': lakehouse_table_path, # Table path for writing delta
    'source_dataframe': df_date,  # DataFrame to upsert
    'match_keys': ['calendar_date_key']  # Key columns to match for upsert
}

# Execute upsert to the dimensional table
upsert_to_table(config)

In [ ]:
# Stage and load operation
df_operation = spark.sql("""
    SELECT DISTINCT
           abs(hash(operationname)) operation_id,
           operationname            operation_name
    FROM vw_stage_metricsbyitemandoperationandhour
    UNION
    SELECT DISTINCT
           abs(hash(operationname)) operation_id,
           operationname            operation_name
    FROM vw_stage_storagebyworkspacesandhour
""")

# Define configuration for the upsert operation
config = {
    'table_name': 'dim_operation',  # Destination table for the upsert
    'lakehouse_table_path': lakehouse_table_path, # Table path for writing delta
    'source_dataframe': df_operation,  # DataFrame to upsert
    'match_keys': ['operation_id']  # Key columns to match for upsert
}

# Execute upsert to the dimensional table
upsert_to_table(config)

In [ ]:
# Stage and load billing type
df_billing_type = spark.sql("""
    SELECT DISTINCT
           abs(hash(billing_type)) billing_type_id,
           billing_type
    FROM vw_stage_storagebyworkspacesandhour
""")

# Define configuration for the upsert operation
config = {
    'table_name': 'dim_billing_type',  # Destination table for the upsert
    'lakehouse_table_path': lakehouse_table_path, # Table path for writing delta
    'source_dataframe': df_billing_type,  # DataFrame to upsert
    'match_keys': ['billing_type_id']  # Key columns to match for upsert
}

# Execute upsert to the dimensional table
upsert_to_table(config)

In [ ]:
# Stage and load workspace storage
df_storage = spark.sql("""
    SELECT UPPER(workspaceid)        workspace_id,
           UPPER(premiumcapacityid)  capacity_id,
           CAST(date AS DATE)        date,
           datetime                  date_time,
           abs(hash(billing_type))   billing_type_id,
           abs(hash(operationname))  operation_id,
           workloadkind              workload_kind,
           storagetype               storage_type,
           staticstorageingb         static_storage_gb,
           CASE WHEN storagetype = 'OneLake Standard Storage'
                THEN staticstorageingb * c.onelake_storage_cost
           END                       static_storage_standard_cost,
           utilization_gb            utilization_gb
    FROM vw_stage_storagebyworkspacesandhour s
    LEFT JOIN vw_dim_capacity c
           ON UPPER(s.premiumcapacityid) = c.capacity_id
""")

# Define configuration for the upsert operation
config = {
    'table_name': 'fact_workspace_storage',  # Destination table for the upsert
    'lakehouse_table_path': lakehouse_table_path, # Table path for writing delta
    'source_dataframe': df_storage,  # DataFrame to upsert
    'match_keys': ['workspace_id', 'date_time', 'capacity_id', 'operation_id']  # Key columns to match for upsert
}

# Execute upsert to the dimensional table
upsert_to_table(config)

In [ ]:
# Stage and load capacity metrics
df_metrics = spark.sql("""
    SELECT UPPER(premiumcapacityid)         capacity_id,
           UPPER(m.workspaceid)             workspace_id,
           UPPER(m.itemid)                  item_id,
           CAST(date AS DATE)               date,
           datetime                         date_time,
           abs(hash(operationname))         operation_id,
           sum_cu                           capacity_unit_seconds_consumed,
           sum_cu / 3600                    capacity_units_consumed,
           (sum_cu / 3600) * (c.pay_go_hour / c.capacity_unit)          activity_cost_pay_go,
           (sum_cu / 3600) * (c.reservation_hour / c.capacity_unit)     activity_cost_reservation,
           sum_duration                     activity_duration,
           count_operations                 total_operations,
           count_users                      users,
           avg_durationms                   average_activity_duration,
           throttling_min                   minutes_throttled,
           count_failure_operations         failed_operations,
           count_rejected_operations        rejected_operations,
           count_successful_operations      successful_operations,
           count_inprogress_operations      in_progress_operations,
           count_cancelled_operations       cancelled_operations,
           count_invalid_operations         invalid_operations
    FROM vw_stage_metricsbyitemandoperationandhour m
    LEFT JOIN vw_dim_capacity c
           ON UPPER(m.premiumcapacityid) = c.capacity_id
    WHERE sum_cu != 0
""")

# Define configuration for the upsert operation
config = {
    'table_name': 'fact_capacity_metrics',  # Destination table for the upsert
    'lakehouse_table_path': lakehouse_table_path, # Table path for writing delta
    'source_dataframe': df_metrics,  # DataFrame to upsert
    'match_keys': ['date_time', 'operation_id', 'item_id', 'workspace_id', 'capacity_id']  # Key columns to match for upsert
}

# Execute upsert to the dimensional table
upsert_to_table(config)

In [ ]:
# Monitoring semantic model
semantic_model = 'Lucid Capacity Monitor'

try:
    # Retrieve the list of datasets
    df_datasets = fabric.list_datasets(fabric.get_workspace_id())
    dataset_names = df_datasets['Dataset Name'].tolist()

    # Check if the semantic model exists
    if semantic_model in dataset_names:
        print(f'Semantic model {semantic_model} already exists. Skipping creation.')
    else:
        import requests
        import json

        # Define .bim file URL
        bim_url = 'https://raw.githubusercontent.com/Lucid-Will/Lucid-Capacity-Monitoring/main/semantic_model_bim/lucid_capacity_monitor.bim'

        # Download and load BIM file as JSON
        response = requests.get(bim_url)
        bim_json = response.json()

        # Deploy new semantic model using the BIM JSON
        fct.create_semantic_model_from_bim(semantic_model, bim_json)
        print(f'Semantic model {semantic_model} created.')

        # Attach semantic model to monitoring lakehouse
        fct.update_direct_lake_model_lakehouse_connection(dataset=semantic_model, workspace=None, lakehouse=lucid_monitor_lakehouse) 

except Exception as e:
    print(f'An unexpected error occurred while processing the semantic model: {e}')